# DBPedia
Get population of countries

In [4]:
%%script bash
arq  '
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
SELECT ?country ?pop
WHERE {
    VALUES ?country {
        dbr:France
        dbr:India
        dbr:Germany
        dbr:Norway
    }
    
    SERVICE <http://dbpedia.org/sparql> {
        ?country dbp:populationCensus | dbo:populationTotal ?pop .
    }
}'

-----------------------------------------------------------------------------------
| country     | pop                                                               |
| dbr:France  | "67081000"^^<http://www.w3.org/2001/XMLSchema#nonNegativeInteger> |
| dbr:India   | 1210854977                                                        |
| dbr:Germany | "83166711"^^<http://www.w3.org/2001/XMLSchema#nonNegativeInteger> |
| dbr:Norway  | "5432580"^^<http://www.w3.org/2001/XMLSchema#nonNegativeInteger>  |
-----------------------------------------------------------------------------------


### Contruct country population using only one property

In [5]:
%%script bash
arq  '
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
CONSTRUCT {
    ?country dbo:populationTotal ?pop
} WHERE {
    VALUES ?country {
        dbr:France
        dbr:India
        dbr:Germany
        dbr:Norway
    }
    
    SERVICE <http://dbpedia.org/sparql> {
        ?country dbp:populationCensus | dbo:populationTotal ?pop .
    }
}'

@prefix dbo:   <http://dbpedia.org/ontology/> .
@prefix dbp:   <http://dbpedia.org/property/> .
@prefix dbr:   <http://dbpedia.org/resource/> .

dbr:Germany  dbo:populationTotal  "83166711"^^<http://www.w3.org/2001/XMLSchema#nonNegativeInteger> .

dbr:India  dbo:populationTotal  1210854977 .

dbr:France  dbo:populationTotal  "67081000"^^<http://www.w3.org/2001/XMLSchema#nonNegativeInteger> .

dbr:Norway  dbo:populationTotal  "5432580"^^<http://www.w3.org/2001/XMLSchema#nonNegativeInteger> .


# WIKIDATA

In [41]:
%%script bash
arq '
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

SELECT  DISTINCT ?countryLabel ?population
WHERE
{
  SERVICE <https://query.wikidata.org/sparql> {
      wd:Q142 wdt:P31/wdt:P279* wd:Q3624078 ; 
               wdt:P1082 ?population .
        wd:Q142 rdfs:label ?countryLabel .
      FILTER (LANG(?countryLabel) = "en")
    }
}
ORDER BY DESC(?population)
'

-------------------------------------------------------------------------
| countryLabel | population                                             |
| "France"@en  | "66628000"^^<http://www.w3.org/2001/XMLSchema#decimal> |
-------------------------------------------------------------------------


08:34:42 WARN  ResponseProcessCookies :: Invalid cookie header: "Set-Cookie: WMF-Last-Access=20-Apr-2021;Path=/;HttpOnly;secure;Expires=Sat, 22 May 2021 00:00:00 GMT". Invalid 'expires' attribute: Sat, 22 May 2021 00:00:00 GMT
08:34:42 WARN  ResponseProcessCookies :: Invalid cookie header: "Set-Cookie: WMF-Last-Access-Global=20-Apr-2021;Path=/;Domain=.wikidata.org;HttpOnly;secure;Expires=Sat, 22 May 2021 00:00:00 GMT". Invalid 'expires' attribute: Sat, 22 May 2021 00:00:00 GMT


# Federated query

In [40]:
%%script bash
arq '
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT DISTINCT ?countryLabel ?population
WHERE
{
  {SERVICE <https://query.wikidata.org/sparql> {
      wd:Q142 wdt:P31/wdt:P279* wd:Q3624078 ; 
               wdt:P1082 ?population .
        wd:Q142 rdfs:label ?countryLabel .
      FILTER (LANG(?countryLabel) = "en")
    }
  } UNION { 
    SERVICE <http://dbpedia.org/sparql> {
       dbr:France  dbp:populationCensus | dbo:populationTotal ?population .
       dbr:France rdfs:label ?countryLabel .
      FILTER (LANG(?countryLabel) = "en")
    }
  }
}
ORDER BY DESC(?population)
'

------------------------------------------------------------------------------------
| countryLabel | population                                                        |
| "France"@en  | "67081000"^^<http://www.w3.org/2001/XMLSchema#nonNegativeInteger> |
| "France"@en  | "66628000"^^<http://www.w3.org/2001/XMLSchema#decimal>            |
------------------------------------------------------------------------------------


08:33:23 WARN  ResponseProcessCookies :: Invalid cookie header: "Set-Cookie: WMF-Last-Access=20-Apr-2021;Path=/;HttpOnly;secure;Expires=Sat, 22 May 2021 00:00:00 GMT". Invalid 'expires' attribute: Sat, 22 May 2021 00:00:00 GMT
08:33:23 WARN  ResponseProcessCookies :: Invalid cookie header: "Set-Cookie: WMF-Last-Access-Global=20-Apr-2021;Path=/;Domain=.wikidata.org;HttpOnly;secure;Expires=Sat, 22 May 2021 00:00:00 GMT". Invalid 'expires' attribute: Sat, 22 May 2021 00:00:00 GMT


# extract subgraph of an ontology

In [12]:
%%script bash
arq --data /Users/marie-angeliquelaporte/Documents/Github/agro/src/ontology/agro.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  
        
SELECT distinct ?uri ?label
WHERE { 
    ?uri rdfs:label ?label . FILTER CONTAINS (?label, "planting implement")
}
' 

-------------------------------------------------------------------------
| uri               | label                                             |
| obo:AGRO_00000535 | "ridging device mounted on planting implement"@en |
| obo:AGRO_00000230 | "planting implement"@en                           |
| obo:AGRO_00000228 | "transplanting implement"@en                      |
-------------------------------------------------------------------------


### full subgraph

In [ ]:
%%script bash
arq --data /Users/marie-angeliquelaporte/Documents/Github/agro/src/ontology/agro.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  
        
CONSTRUCT {
    ?s ?p ?o
}
WHERE { 
    obo:AGRO_00000230 (owl:|!owl:)* ?s .
    ?s ?p ?o .
     
}
' 

### only label and definition subgraph

In [66]:
%%script bash
arq --data /Users/marie-angeliquelaporte/Documents/Github/agro/src/ontology/agro.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  
        
CONSTRUCT {
    ?s rdfs:label ?label .
    ?s rdfs:comment ?definition .
    ?s rdf:type owl:Class
}
WHERE { 
    ?s rdfs:subClassOf* obo:AGRO_00000230 .
    ?s rdfs:label ?label .
    ?s obo:IAO_0000115 ?definition . 
}
' 

@prefix :      <http://purl.obolibrary.org/obo/agro-edit.owl#> .
@prefix oboInOwl: <http://www.geneontology.org/formats/oboInOwl#> .
@prefix owl:   <http://www.w3.org/2002/07/owl#> .
@prefix obo1:  <http://purl.obolibrary.org/obo/> .
@prefix xsd:   <http://www.w3.org/2001/XMLSchema#> .
@prefix swrl:  <http://www.w3.org/2003/11/swrl#> .
@prefix swrlb: <http://www.w3.org/2003/11/swrlb#> .
@prefix rdfs:  <http://www.w3.org/2000/01/rdf-schema#> .
@prefix swrla: <http://swrl.stanford.edu/ontologies/3.3/swrla.owl#> .
@prefix oio:   <http://www.geneontology.org/formats/oboInOwl#> .
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix obo:   <http://purl.obolibrary.org/obo/> .
@prefix dc:    <http://purl.org/dc/elements/1.1/> .

obo:AGRO_00000398  a  owl:Class ;
        rdfs:comment  "An agricultural hand tool ade of a blade set to an handle and used to shape the soil, control weeds, clear soil, and harvest root crops."@en ;
        rdfs:label    "hoe"@en .

obo:AGRO_00000402